In [1]:
import os
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import figure_setup as fs

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [2]:
# G12 line in G1G2 space
def G12_to_G1G2(g12):
    G1_a, G2_a = 0.0, 0.53513350
    G1_b, G2_b = 0.84293649, -0.53513350

    G1 = G1_a + g12 * G1_b
    G2 = G2_a + g12 * G2_b

    return G1, G2


# Simple 1D Gaussian
def gauss(x, m, mu, sigma):
    return m * np.exp(-((x - mu) ** 2) / (2 * sigma**2))


# Two 1D Gaussians
def double_gauss(x, m1, mu1, sigma1, m2, mu2, sigma2):
    g1 = gauss(x, m1, mu1, sigma1)
    g2 = gauss(x, m2, mu2, sigma2)
    return g1 + g2

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [4]:
# ZTF filters
filters = {"1": "g", "2": "r"}

# S_color = {'g': -0.3928, 'r':0.2913}
# sun_color = {'g': -0.3044, 'r':0.1903}

# color_C = -(sun_color['g']-sun_color['r'])
# color_S = -(S_color['g']-S_color['r'])


# mag_to_V ={
#         1: -0.2833,  # ZTF g
#         2: 0.1777,   # ZTF r
#         3: 0.4388,   # ATLAS orange
#         4: -0.0986   # ATLAS cyan
#     }

fink_colors = ["#15284F", "#F5622E"]

In [5]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [6]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)


data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [7]:
len(data)

122675

# Compare results

In [8]:
data.HG1G2_status.value_counts()

HG1G2_status
 2.0    113644
 4.0      6886
 1.0      1320
 3.0       521
-2.0       301
 0.0         3
Name: count, dtype: int64

In [9]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



In [39]:
mask = mask_HG1G2_inter & mask_SHG1G2_inter

## Figure G1G2

In [40]:
data.columns[:80]

Index(['ssnamenr', 'HG_chi2red', 'HG_status', 'HG_fit', 'HG_rms', 'HG_rms_g',
       'HG_rms_r', 'HG_median_error_phot', 'HG_median_error_phot_1',
       'HG_median_error_phot_2', 'HG_H_g', 'HG_dH_g', 'HG_G_g', 'HG_dG_g',
       'HG_H_r', 'HG_dH_r', 'HG_G_r', 'HG_dG_r', 'HG_flag', 'HG1G2_chi2red',
       'HG1G2_status', 'HG1G2_fit', 'HG1G2_rms', 'HG1G2_rms_g', 'HG1G2_rms_r',
       'HG1G2_median_error_phot', 'HG1G2_median_error_phot_1',
       'HG1G2_median_error_phot_2', 'HG1G2_H_g', 'HG1G2_dH_g', 'HG1G2_G1_g',
       'HG1G2_dG1_g', 'HG1G2_G2_g', 'HG1G2_dG2_g', 'HG1G2_H_r', 'HG1G2_dH_r',
       'HG1G2_G1_r', 'HG1G2_dG1_r', 'HG1G2_G2_r', 'HG1G2_dG2_r', 'HG1G2_flag',
       'SHG1G2_chi2red', 'SHG1G2_min_cos_lambda', 'SHG1G2_mean_cos_lambda',
       'SHG1G2_max_cos_lambda', 'SHG1G2_status', 'SHG1G2_fit', 'SHG1G2_rms',
       'SHG1G2_rms_g', 'SHG1G2_rms_r', 'SHG1G2_median_error_phot',
       'SHG1G2_median_error_phot_1', 'SHG1G2_median_error_phot_2', 'n_obs',
       'n_obs_g', 'n_obs_r', 

In [41]:
# --------------------------------------------------------------------------------
# Fig + axes
fig, ax = plt.subplots(
    2,
    2,
    figsize=fs.figsize(0.5),
#     gridspec_kw={
#         "left": 0.11,
#         "right": 0.98,
#         # "bottom": 0.05,
#         "bottom": 0.12,
#         "top": 0.98,
#         "hspace": 0.02,
#         "wspace": 0.01,
#     },
)

r = [0, 1]
b = 100

mm = mask

# --------------------------------------------------------------------------------
# G1
ax[0, 0].hist2d(
    data.HG1G2_G1_g[mm],
    data.SHG1G2_G1_g[mm],
    rasterized=True,
    density=True,
    range=[r, r],
    bins=b,
    cmap="Blues",
    norm=mpl.colors.LogNorm(),
)
ax[0, 1].hist2d(
    data.HG1G2_G1_r[mm],
    data.SHG1G2_G1_r[mm],
    rasterized=True,
    density=True,
    range=[r, r],
    bins=b,
    cmap="Oranges",
    norm=mpl.colors.LogNorm(),
)

# --------------------------------------------------------------------------------
# G2
ax[1, 0].hist2d(
    data.HG1G2_G2_g[mm],
    data.SHG1G2_G2_g[mm],
    rasterized=True,
    density=True,
    range=[r, r],
    bins=b,
    cmap="Blues",
    norm=mpl.colors.LogNorm(),
)
ax[1, 1].hist2d(
    data.HG1G2_G2_r[mm],
    data.SHG1G2_G2_r[mm],
    rasterized=True,
    density=True,
    range=[r, r],
    bins=b,
    cmap="Oranges",
    norm=mpl.colors.LogNorm(),
)


for a in ax.ravel():
    xx = np.linspace(0,1,num=10)
    a.plot(xx, xx, color='grey', linestyle='dashed')


# --------------------------------------------------------------------------------
# Axes
for a in ax[:, 1:].ravel():
    a.set_yticklabels("")
for a in ax[0, :].ravel():
    a.set_xticklabels("")

# X ticks
for a in ax[1,:].ravel():
    a.set_xticks([0,0.25,0.5,0.75,1])
ax[1,0].set_xticklabels(['0','0.25','0.5','0.75','1.0'])
ax[1,1].set_xticklabels(['','0.25','0.5','0.75','1.0'])

# Y ticks
for a in ax[1:,0].ravel():
    a.set_yticks([0,0.25,0.5,0.75,1])
ax[0,0].set_yticklabels([' ','0.25','0.50','0.75','1.00'])
ax[1,0].set_yticklabels(['0','0.25','0.50','0.75','1.00'])

# Labels
ax[0, 0].set_ylabel("$G_1$ ($\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)")
ax[0, 0].set_xlabel("$G_1$ ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)", labelpad=0)

ax[1, 0].set_ylabel("$G_2$ ($\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)")
ax[1, 0].set_xlabel("$G_2$ ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)", labelpad=0)

ax[0, 1].set_ylabel("$G_1$ ($\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)")
ax[0, 1].set_xlabel("$G_1$ ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)", labelpad=0)

ax[1, 1].set_ylabel("$G_2$ ($\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)")
ax[1, 1].set_xlabel("$G_2$ ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)", labelpad=0)

# ax[1, 0].set_ylabel("$G_2$")

ax[0, 0].set_title("$g$ filter", color='C0')
ax[0, 1].set_title("$r$ filter", color='C1')


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "g1g2_comparison.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "g1g2_comparison.pgf"),
)
plt.close()

/tmp/ipykernel_715402/1788196748.py:91: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax[0,0].set_yticklabels([' ','0.25','0.50','0.75','1.00'])
